In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("dataset.csv")
# Display the first few rows of the DataFrame
print(df.head())

           frame_path  q_state  q_timestamp  q_hold_duration  w_state   
0  frames/frame_0.png        0          NaN              0.0        0  \
1  frames/frame_1.png        0          NaN              0.0        0   
2  frames/frame_2.png        0          NaN              0.0        0   
3  frames/frame_3.png        0          NaN              0.0        0   
4  frames/frame_4.png        0          NaN              0.0        0   

   w_timestamp  w_hold_duration  e_state  e_timestamp  e_hold_duration  ...   
0          NaN              0.0        0          NaN              0.0  ...  \
1          NaN              0.0        0          NaN              0.0  ...   
2          NaN              0.0        0          NaN              0.0  ...   
3          NaN              0.0        0          NaN              0.0  ...   
4          NaN              0.0        0          NaN              0.0  ...   

   up_hold_duration  down_state  down_timestamp  down_hold_duration   
0              

In [3]:
# Load the image data

from PIL import Image
import numpy as np
df['frame_path'] = df['frame_path'].apply(lambda x: np.array(Image.open(x)))

In [4]:
# normalize the image data
df['frame_path'] = df['frame_path'].apply(lambda x: x/255)

In [5]:
# Columns to be dropped
drop_columns = ['q_timestamp', 'q_hold_duration',
                'w_timestamp', 'w_hold_duration',
                'e_timestamp', 'e_hold_duration',
                'r_timestamp', 'r_hold_duration',
                'up_timestamp', 'up_hold_duration',
                'down_timestamp', 'down_hold_duration',
                'left_timestamp', 'left_hold_duration',
                'right_timestamp', 'right_hold_duration']

# Drop the columns
df = df.drop(columns=drop_columns)

print(df.head())


                                          frame_path  q_state  w_state   
0  [[[0.5098039215686274, 0.611764705882353, 0.68...        0        0  \
1  [[[0.5254901960784314, 0.615686274509804, 0.71...        0        0   
2  [[[0.5254901960784314, 0.615686274509804, 0.71...        0        0   
3  [[[0.5529411764705883, 0.6627450980392157, 0.7...        0        0   
4  [[[0.5372549019607843, 0.6470588235294118, 0.7...        0        0   

   e_state  r_state  up_state  down_state  left_state  right_state  
0        0        0         0           0           0            0  
1        0        0         0           0           0            0  
2        0        0         0           0           0            0  
3        0        0         0           0           0            0  
4        0        0         0           0           0            0  


In [6]:
# itterate through rows to check if no key state is 1 and drop the row
for index, row in df.iterrows():
    if row['q_state'] == 0 and row['w_state'] == 0 and row['e_state'] == 0 and row['r_state'] == 0 and row['up_state'] == 0 and row['down_state'] == 0 and row['left_state'] == 0 and row['right_state'] == 0:
        df = df.drop(index)
        
print(df.head())

                                           frame_path  q_state  w_state   
70  [[[0.5333333333333333, 0.6431372549019608, 0.7...        0        0  \
71  [[[0.5333333333333333, 0.6431372549019608, 0.7...        0        0   
72  [[[0.5333333333333333, 0.6431372549019608, 0.7...        0        0   
73  [[[0.5333333333333333, 0.6431372549019608, 0.7...        0        0   
74  [[[0.5333333333333333, 0.6431372549019608, 0.7...        0        0   

    e_state  r_state  up_state  down_state  left_state  right_state  
70        0        0         1           0           0            1  
71        0        0         1           0           0            1  
72        0        0         1           0           0            1  
73        0        0         1           0           0            0  
74        0        0         1           0           1            0  


In [7]:
df.describe()

,q_state,w_state,e_state,r_state,up_state,down_state,left_state,right_state
count,1993.000000,1993.0,1993.000000,1993.000000,1993.000000,1993.0,1993.000000,1993.000000
mean,0.153537,0.0,0.026593,0.000502,0.983442,0.0,0.182639,0.180632
std,0.360595,0.0,0.160931,0.022400,0.127640,0.0,0.386467,0.384810
min,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
25%,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
50%,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
75%,0.000000,0.0,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
max,1.000000,0.0,1.000000,1.000000,1.000000,0.0,1.000000,1.000000


In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
# Define features (X) and labels (Y) for training set
X_train = train_data['frame_path']
Y_train = train_data.drop('frame_path', axis=1)

# Define features (X) and labels (Y) for testing set
X_test = test_data['frame_path']
Y_test = test_data.drop('frame_path', axis=1)

In [10]:
X_train = np.stack(train_data['frame_path'])
X_test = np.stack(test_data['frame_path'])

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1:])),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(Y_train.shape[1], activation='sigmoid')   # number of output neurons equals to number of key states
])


In [12]:

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy',   # use binary_crossentropy for multi-label classification
              metrics=['accuracy'])

# Print a summary of the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 636, 814, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 318, 407, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 318, 407, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 316, 405, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 158, 202, 64)     0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 158, 202, 64)      0

In [13]:
# Train the model
history = model.fit(X_train, Y_train, validation_split=0.2, epochs=5, batch_size=1)

Epoch 1/5
1275/1275 [==============================] - 71s 50ms/step - loss: 0.2591 - accuracy: 0.8094 - val_loss: 0.1758 - val_accuracy: 0.8088
Epoch 2/5
1275/1275 [==============================] - 55s 43ms/step - loss: 0.1672 - accuracy: 0.8063 - val_loss: 0.1528 - val_accuracy: 0.8088
Epoch 3/5
1275/1275 [==============================] - 54s 43ms/step - loss: 0.1410 - accuracy: 0.8008 - val_loss: 0.1635 - val_accuracy: 0.8025
Epoch 4/5
1275/1275 [==============================] - 55s 43ms/step - loss: 0.1195 - accuracy: 0.8039 - val_loss: 0.1505 - val_accuracy: 0.7994
Epoch 5/5
1275/1275 [==============================] - 55s 43ms/step - loss: 0.0860 - accuracy: 0.8102 - val_loss: 0.1399 - val_accuracy: 0.8401


In [14]:
# model.save('model.h5')

In [11]:
import keras
load_model = keras.models.load_model('model.h5')

In [13]:
# Predict on the test set
predictions = load_model.predict(X_test)

13/13 [==============================] - 7s 120ms/step


In [14]:
print(predictions[0])

[1.2958232e-02 1.2400840e-08 2.7755898e-04 5.8167950e-07 9.9930668e-01
 1.9377302e-09 4.2076878e-02 8.5442655e-02]


In [15]:
print(Y_test.iloc[0])

q_state        0
w_state        0
e_state        0
r_state        0
up_state       1
down_state     0
left_state     0
right_state    0
Name: 965, dtype: int64
